In [1]:
from osgeo import ogr
import os

In [6]:
census_fn = os.path.join('data', 'ca_census_albers.shp')
census_ds = ogr.Open(census_fn, True)
census_lyr = census_ds.GetLayer()
density_field = ogr.FieldDefn('popsqkm', ogr.OFTReal)
census_lyr.CreateField(density_field)
for row in census_lyr:
    pop = row.GetField('HD01_S001')
    sqkm = row.geometry().GetArea() / 1000000
    row.SetField('popsqkm', pop / sqkm)
    census_lyr.SetFeature(row)

Warning 6: Normalized/laundered field name: 'popsqkm' to 'popsqkm_3'


In [13]:
#Listing9.1 create my first tif file!!
import os
from osgeo import gdal
from pathlib import Path

base=Path("sentinel")
b2=base/"T54STD_20230306T013659_B02_60m.jp2"
b3=base/"T54STD_20230306T013659_B03_60m.jp2"
b4=b2=base/"T54STD_20230306T013659_B04_60m.jp2"

in_ds=gdal.Open(str(b2))
in_band=in_ds.GetRasterBand(1)
print(type(in_ds))
print(type(in_band))
print(in_ds.RasterCount)
print(in_ds.RasterXSize)
print(in_band.XSize)


<class 'osgeo.gdal.Dataset'>
<class 'osgeo.gdal.Band'>
1
1830
1830


AttributeError: 'Band' object has no attribute 'RasterXsize'

In [5]:
# tiff作成(ファイル名、配列サイズ、,種類(8bit画像とか))
#srsの投入
#originの情報と1ピクセルの縮尺
#各バンドの配列を各バンドに放り込むｓ

driver=gdal.GetDriverByName("GTiff")
out_ds=driver.Create("nat_colof.tif",in_band.XSize,in_band.YSize,3,in_band.DataType)
out_ds.SetProjection(in_ds.GetProjection())
print(in_ds.GetProjection())
print(in_ds.GetGeoTransform())
print(in_band.DataType)
out_ds.SetGeoTransform(in_ds.GetGeoTransform())

PROJCS["WGS 84 / UTM zone 54N",GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS 84",6378137,298.257223563,AUTHORITY["EPSG","7030"]],AUTHORITY["EPSG","6326"]],PRIMEM["Greenwich",0,AUTHORITY["EPSG","8901"]],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]],AUTHORITY["EPSG","4326"]],PROJECTION["Transverse_Mercator"],PARAMETER["latitude_of_origin",0],PARAMETER["central_meridian",141],PARAMETER["scale_factor",0.9996],PARAMETER["false_easting",500000],PARAMETER["false_northing",0],UNIT["metre",1,AUTHORITY["EPSG","9001"]],AXIS["Easting",EAST],AXIS["Northing",NORTH],AUTHORITY["EPSG","32654"]]
(199980.0, 60.0, 0.0, 3900000.0, 0.0, -60.0)
2


0

In [10]:
in_data=in_band.ReadAsArray()
out_band=out_ds.GetRasterBand(3)
out_band.WriteArray(in_data)

0

In [12]:
in_ds=gdal.Open(str(b2))
out_band=out_ds.GetRasterBand(2)
out_band.WriteArray(in_ds.ReadAsArray())

0

In [14]:
out_ds.GetRasterBand(1).WriteArray(gdal.Open(str(b3)).ReadAsArray())
out_ds.FlushCache()

In [17]:
for i in range(1, 4):
    out_ds.GetRasterBand(i).ComputeStatistics(False)

# Build overview layers for faster display.
out_ds.BuildOverviews('average', [2, 4, 8, 16, 32])
del out_ds

In [21]:
in_ds.GetRasterBand(1)
in_band.GetBlockSize()
print(in_band.GetNoDataValue())

None


In [1]:
# realworld coordinate


In [11]:
import os
from osgeo import gdal

in_ds=gdal.Open("data/nat_color.tif")
in_gt=in_ds.GetGeoTransform()

#xの原点の座標、回転,ピクセルサイズ、
#yの原点の座標、、回転,ピクセルサイズ(北上ならnegatives)
print(in_gt)
print(in_ds.RasterCount)

inv=gdal.InvGeoTransform(in_gt)
#画像でグロ塗りの部分を調べてみる
offsets=gdal.ApplyGeoTransform(inv,500832,5274316)#357107,5341747)
x,y=map(int,offsets)
print(in_ds.ReadAsArray(x,y,1,1))


(343724.25, 28.5, 0.0, 5369585.25, 0.0, -28.5)
4
[[[ 31]]

 [[ 46]]

 [[ 57]]

 [[255]]]


In [12]:
import os
from osgeo import gdal

# Coordinates for the bounding box to extract.
vashon_ulx, vashon_uly = 532000, 5262600
vashon_lrx, vashon_lry = 548500, 5241500

# Don't forget to change the directory.
#')
in_ds = gdal.Open('data/nat_color.tif')

# Create an inverse geotransform for the raster. This converts real-world
# coordinates to pixel offsets.
in_gt = in_ds.GetGeoTransform()
inv_gt = gdal.InvGeoTransform(in_gt)
if gdal.VersionInfo()[0] == '1':
    if inv_gt[0] == 1:
        inv_gt = inv_gt[1]
    else:
        raise RuntimeError('Inverse geotransform failed')
elif inv_gt is None:
    raise RuntimeError('Inverse geotransform failed')

# Get the offsets that correspond to the bounding box corner coordinates.
offsets_ul = gdal.ApplyGeoTransform(
    inv_gt, vashon_ulx, vashon_uly)
offsets_lr = gdal.ApplyGeoTransform(
    inv_gt, vashon_lrx, vashon_lry)

# The offsets are returned as floating point, but we need integers.
off_ulx, off_uly = map(int, offsets_ul)
off_lrx, off_lry = map(int, offsets_lr)

# Compute the numbers of rows and columns to extract, based on the offsets.
rows = off_lry - off_uly
columns = off_lrx - off_ulx

# Create an output raster with the correct number of rows and columns.
gtiff_driver = gdal.GetDriverByName('GTiff')
out_ds = gtiff_driver.Create('data/vashon.tif', columns, rows, 3)
out_ds.SetProjection(in_ds.GetProjection())

# Convert the offsets to real-world coordinates for the georeferencing info.
# We can't use the coordinates above because they don't correspond to the
# pixel edges.
subset_ulx, subset_uly = gdal.ApplyGeoTransform(
    in_gt, off_ulx, off_uly)
out_gt = list(in_gt)
out_gt[0] = subset_ulx
out_gt[3] = subset_uly
out_ds.SetGeoTransform(out_gt)

# Loop through the 3 bands.
for i in range(1, 4):
    in_band = in_ds.GetRasterBand(i)
    out_band = out_ds.GetRasterBand(i)

    # Read the data from the input raster starting at the computed offsets.
    data = in_band.ReadAsArray(
        off_ulx, off_uly, columns, rows)

    # Write the data to the output, but no offsets are needed because we're
    # filling the entire image.
    out_band.WriteArray(data)

del out_ds